# 前処理

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels.panel import PanelOLS

In [2]:
from replace import Country
country = Country()

pd.options.display.float_format = '{:.4f}'.format

visitors = pd.read_csv('data/visitors.csv')
gdp = pd.read_csv('data/gdp.csv')
currency = pd.read_csv('data/currency.csv')
continent = pd.read_csv('data/continent.csv')
cpi = pd.read_csv('data/cpi_japan.csv')
distance = pd.read_csv('data/distance.csv')
population = pd.read_csv('data/population.csv')

In [3]:
visitors["Country"] = visitors["Country"].replace(country.replace_list)
visitors = visitors.loc[~visitors["Country"].isin(country.list)]
visitors = visitors.loc[visitors["Year"] < 2020]

gdp = gdp.melt(id_vars="Country", var_name="Year", value_name="GDP")
gdp["Year"] = gdp.Year.astype(int)

population = population.melt(id_vars="Country", var_name="Year", value_name="Population")
population["Year"] = population.Year.astype(int)

In [4]:
df = pd.merge(visitors, currency, on=["Country", "Year"], how="inner")
df = pd.merge(df, gdp, on=["Country", "Year"], how="inner")
df = pd.merge(df, cpi, on=["Country", "Year"], how="inner")
df = pd.merge(df, population, on=["Country", "Year"], how="inner")

In [5]:
cluster1 = [
        'Australia', 'Austria', 'Belgium', 'Canada', 'Denmark', 'Finland',
        'France', 'Germany', 'Ireland', 'Israel', 'Italy', 'Netherlands',
        'New Zealand', 'Norway', 'Portugal', 'Singapore', 'Spain',
        'Sweden', 'Switzerland', 'United Kingdom', 'United States'
        ]

cluster2 = [
        'China', 'Hong Kong', 'Korea', 'Malaysia', 'Philippines', 'Poland',
        'Taiwan', 'Thailand'
        ]

cluster3 = ['Brazil', 'India', 'Indonesia', 'Mexico', 'Russia', 'Turkey', 'Vietnam']

In [6]:
df["Cluster"] = 0
df.loc[df["Country"].isin(cluster1), "Cluster"] = 1
df.loc[df["Country"].isin(cluster2), "Cluster"] = 2
df.loc[df["Country"].isin(cluster3), "Cluster"] = 3

In [7]:
df["RER"] = df["Exchange Rate"] * df["CPI_Japan"] / df["CPI"]
df["GDPpc"] = df["GDP"] / df["Population"]
df["log_Visitors"] = np.log(df["Visitors"])
df["Visitors/Population"] = df["Visitors"] / df["Population"]
df["log_Visitors/Population"] = np.log(df["Visitors/Population"])
df["log_GDP"] = np.log(df["GDP"])
df["log_GDPpc"] = np.log(df["GDPpc"])
df["log_RER"] = np.log(df["RER"])

In [8]:
df = pd.concat([df, pd.get_dummies(df["Year"], prefix="yr", prefix_sep="", drop_first=True, dtype=int)], axis=1)

In [9]:
select = [
        'Year', 'Country', 'Cluster',
        'log_Visitors', 'log_Visitors/Population',
        'log_GDP', 'log_GDPpc', 'log_RER',
        'yr2011', 'yr2012', 'yr2013',
        'yr2014', 'yr2015', 'yr2016',
        'yr2017', 'yr2018', 'yr2019'
]

df = df[select]

In [10]:
df_cluster1 = df.copy()
df_cluster2 = df.copy()
df_cluster3 = df.copy()
df_cluster4 = df.copy()

In [11]:
df_cluster1 = df_cluster1.loc[df["Cluster"]==1]
df_cluster2 = df_cluster2.loc[df["Cluster"]==2]
df_cluster3 = df_cluster3.loc[df["Cluster"]==3]
df_cluster4 = df_cluster4.loc[df["Cluster"]==4]

# PanelOLS

In [12]:
df_cluster1["Year"] = pd.to_datetime(df_cluster1["Year"], format="%Y")
df_cluster1.set_index(["Country", "Year"], inplace=True)

In [13]:
model = PanelOLS(df_cluster1["log_Visitors"], df_cluster1[['log_GDP', 'log_RER',
                                                                        'yr2011', 'yr2012', 'yr2013',
                                                                        'yr2014', 'yr2015', 'yr2016',
                                                                        'yr2017', 'yr2018', 'yr2019']],
                 entity_effects=True,
                 time_effects=False)
results = model.fit()
print(results.summary.tables[1])

                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
log_GDP        0.1725     0.1626     1.0606     0.2903     -0.1484      0.4933
log_RER       -0.3596     0.1754    -2.0501     0.0418     -0.7057     -0.0135
yr2011        -0.4209     0.0320    -13.142     0.0000     -0.4841     -0.3577
yr2012        -0.1000     0.0327    -3.0632     0.0025     -0.1645     -0.0356
yr2013        -0.0174     0.0468    -0.3729     0.7096     -0.1098      0.0749
yr2014         0.1243     0.0540     2.3003     0.0226      0.0177      0.2309
yr2015         0.3244     0.0514     6.3075     0.0000      0.2229      0.4258
yr2016         0.5010     0.0404     12.386     0.0000      0.4212      0.5808
yr2017         0.5534     0.0475     11.648     0.0000      0.4596      0.6472
yr2018         0.6547     0.0507     12.909     0.00

In [14]:
df_cluster2["Year"] = pd.to_datetime(df_cluster2["Year"], format="%Y")
df_cluster2.set_index(["Country", "Year"], inplace=True)

In [15]:
model = PanelOLS(df_cluster2["log_Visitors"], df_cluster2[['log_GDP', 'log_RER',
                                                                        'yr2011', 'yr2012', 'yr2013',
                                                                        'yr2014', 'yr2015', 'yr2016',
                                                                        'yr2017', 'yr2018', 'yr2019']],
                 entity_effects=True,
                 time_effects=False)
results = model.fit()
print(results.summary.tables[1])

                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
log_GDP        1.7651     0.3674     4.8047     0.0000      1.0305      2.4997
log_RER       -0.6069     0.3252    -1.8663     0.0668     -1.2572      0.0433
yr2011        -0.3808     0.0759    -5.0146     0.0000     -0.5326     -0.2289
yr2012        -0.1084     0.0809    -1.3405     0.1851     -0.2701      0.0533
yr2013        -0.0146     0.1203    -0.1212     0.9040     -0.2551      0.2260
yr2014         0.1920     0.1422     1.3505     0.1819     -0.0923      0.4763
yr2015         0.4425     0.1666     2.6567     0.0101      0.1094      0.7756
yr2016         0.6612     0.1477     4.4769     0.0000      0.3659      0.9565
yr2017         0.6825     0.1697     4.0206     0.0002      0.3431      1.0219
yr2018         0.6927     0.1847     3.7503     0.00

In [16]:
df_cluster3["Year"] = pd.to_datetime(df_cluster3["Year"], format="%Y")
df_cluster3.set_index(["Country", "Year"], inplace=True)

In [17]:
model = PanelOLS(df_cluster3["log_Visitors"], df_cluster3[['log_GDP', 'log_RER',
                                                                        'yr2011', 'yr2012', 'yr2013',
                                                                        'yr2014', 'yr2015', 'yr2016',
                                                                        'yr2017', 'yr2018', 'yr2019']],
                 entity_effects=True,
                 time_effects=False)
results = model.fit()
print(results.summary.tables[1])

                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
log_GDP        0.5339     0.3822     1.3969     0.1684     -0.2330      1.3008
log_RER       -1.3632     0.2891    -4.7151     0.0000     -1.9433     -0.7830
yr2011        -0.2451     0.1056    -2.3209     0.0242     -0.4570     -0.0332
yr2012         0.1289     0.1115     1.1566     0.2527     -0.0947      0.3526
yr2013        -0.0075     0.1143    -0.0657     0.9479     -0.2368      0.2218
yr2014         0.1340     0.1197     1.1194     0.2681     -0.1062      0.3742
yr2015         0.2800     0.1236     2.2644     0.0277      0.0319      0.5281
yr2016         0.5714     0.1324     4.3166     0.0001      0.3058      0.8371
yr2017         0.6569     0.1389     4.7306     0.0000      0.3782      0.9355
yr2018         0.8440     0.1527     5.5260     0.00